# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 19

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,90s,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,30-148,1990.0,10,9,829,9.2,0,1,0,0,1,0,0.00,0.11,0.11,0.00,0.11,0.7,0.7,0.7,0.08,0.08,0.16,0.08,0.16,Matches
2,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,23-114,1997.0,16,9,898,10.0,6,1,0,0,0,0,0.60,0.10,0.70,0.60,0.70,5.5,5.5,0.9,0.55,0.09,0.64,0.55,0.64,Matches
3,Che Adams\Che-Adams,eng ENG,FW,Southampton,24-195,1996.0,17,17,1417,15.7,4,4,0,0,1,0,0.25,0.25,0.51,0.25,0.51,3.7,3.7,3.9,0.23,0.25,0.48,0.23,0.48,Matches
4,Tosin Adarabioyo\Tosin-Adarabioyo,eng ENG,DF,Fulham,23-122,1997.0,14,14,1260,14.0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.6,0.6,0.1,0.04,0.01,0.05,0.04,0.05,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,34-021,1987.0,2,2,180,2.0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col='Squad')
teamStats.head()

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Manchester Utd,1,19,12,4,3,36,25,11,40,30.8,24.7,6.1,0.32,W W W D W,NaN,Bruno Fernandes - 11,David de Gea,NaN
Manchester City,2,18,11,5,2,31,13,18,38,33.3,14.3,19.0,1.05,W W W W W,NaN,İlkay Gündoğan Raheem Sterling - 5,Ederson,NaN
Leicester City,3,19,12,2,5,35,21,14,38,28.4,21.6,6.8,0.36,D D W W W,NaN,Jamie Vardy - 11,Kasper Schmeichel,NaN
Liverpool,4,19,9,7,3,37,22,15,34,37.1,22.3,14.8,0.78,D D L D L,600.0,Mohamed Salah - 13,Alisson,NaN
Tottenham,5,18,9,6,3,33,17,16,33,26.3,19.3,6.9,0.39,L D W D W,222.0,Harry Kane Son Heung-min - 12,Hugo Lloris,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-3,3,0.0,641,257,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,641,257,False,0.0,641,257,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,67,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.5,False,None,u,1,3,0.0,641,257,0,3441,3,53623,192,0.0,0.0,Özil,0
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-2,2,0.0,589,219,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,602,219,False,0.0,599,219,0,Left the club by mutual consent on 20/1,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,None,u,1,3,0.0,582,215,0,10266,1,18947,22,0.0,0.0,Sokratis,0
3,0,0,108,100.0,100.0,2,41270,NaN,,0,0,-1,1,28.8,312,100,4.0,,0,2,2.7,2.7,6,David,2.2,9,0,20.0,298,98,False,115.4,259,99,659,,2020-12-26T18:00:15.638627Z,54,0,0,NaN,0,,41270.jpg,2.0,0,0,Luiz Moreira Marinho,0.8,False,None,a,1,3,56.0,251,77,20,43007,1128,92700,751,0.4,3.7,David Luiz,1
4,1,6,202,75.0,100.0,7,54694,NaN,,0,0,-7,7,226.3,70,49,NaN,,1,3,4.0,5.3,15,Pierre-Emerick,4.8,16,5,101.8,32,22,False,272.2,114,39,1514,Personal Reasons - 75% chance of playing,2021-01-23T16:00:22.555453Z,113,1,0,1.0,0,,54694.jpg,4.2,0,0,Aubameyang,7.2,False,None,d,1,3,519.0,20,12,71,557854,34685,3206687,13713,0.4,6.3,Aubameyang,2
5,1,3,42,100.0,100.0,1,58822,NaN,,0,0,-4,4,32.3,302,93,NaN,,0,2,2.9,2.9,12,Cédric,2.4,0,0,6.4,387,145,False,29.2,381,151,95,,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,,58822.jpg,6.5,0,0,Soares,0.2,False,None,a,1,3,2.0,400,151,13,5929,1594,35713,436,0.5,2.8,Cédric,0


In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats.index)
# for some reason the fpl team numbers are not alphabetical with leeds and leicester...
if (team_names[8]=='Leeds United') & (team_names[9]=='Leicester City'):
    team_names[8] = 'Leicester City'
    team_names[9] = 'Leeds United'
df['team_name'] = team_names[df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [7]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Manchester Utd,1,19,12,4,3,36,25,11,40,30.8,24.7,6.1,0.32,W W W D W,NaN,Bruno Fernandes - 11,David de Gea,NaN,1.621053,1.300000,0.272532
Manchester City,2,18,11,5,2,31,13,18,38,33.3,14.3,19.0,1.05,W W W W W,NaN,İlkay Gündoğan Raheem Sterling - 5,Ederson,NaN,1.850000,0.794444,0.451832
Leicester City,3,19,12,2,5,35,21,14,38,28.4,21.6,6.8,0.36,D D W W W,NaN,Jamie Vardy - 11,Kasper Schmeichel,NaN,1.494737,1.136842,0.320831
Liverpool,4,19,9,7,3,37,22,15,34,37.1,22.3,14.8,0.78,D D L D L,600.0,Mohamed Salah - 13,Alisson,NaN,1.952632,1.173684,0.309226
Tottenham,5,18,9,6,3,33,17,16,33,26.3,19.3,6.9,0.39,L D W D W,222.0,Harry Kane Son Heung-min - 12,Hugo Lloris,NaN,1.461111,1.072222,0.342247
Everton,6,17,10,2,5,28,21,7,32,21.6,21.2,0.4,0.02,W W W L W,250.0,Dominic Calvert-Lewin - 11,Jordan Pickford,NaN,1.270588,1.247059,0.287349
West Ham,7,19,9,5,5,27,22,5,32,24.7,22.5,2.2,0.12,D D W W W,200.0,Michail Antonio Tomáš Souček... - 5,Łukasz Fabiański,NaN,1.300000,1.184211,0.305988
Aston Villa,8,17,9,2,6,31,18,13,29,29.8,22.3,7.5,0.44,W D L L W,NaN,Ollie Watkins - 7,Emiliano Martínez,NaN,1.752941,1.311765,0.269344
Chelsea,9,19,8,5,6,33,23,10,29,30.6,20.7,9.8,0.52,L D L W L,222.0,Tammy Abraham - 6,Edouard Mendy,NaN,1.610526,1.089474,0.336393


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [8]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
xA_proportion

0.7015624999999999

In [9]:
df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
if latest_gameweek == 1:
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,xGA_week12,cleansheet_week12,xA_week12,assists_week12,xPoints week 13,xG_week13,goals_week13,xGA_week13,cleansheet_week13,xA_week13,assists_week13,xPoints week 14,xG_week14,goals_week14,xGA_week14,cleansheet_week14,xA_week14,assists_week14,xPoints week 15,xG_week15,goals_week15,xGA_week15,cleansheet_week15,xA_week15,assists_week15,xPoints week 16,xG_week16,goals_week16,xGA_week16,cleansheet_week16,xA_week16,assists_week16,xPoints week 17,xG_week17,goals_week17,xGA_week17,cleansheet_week17,xA_week17,assists_week17,form 5,form 10,next1_xP,next5_xP,next10_xP,value,value form 5,value form 10,valuePoints metric,xG+xA,xG+xA per game
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,NaN,0,0,-2,2,0.0,633,254,NaN,NaN,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,633,254,False,0.0,633,254,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,NaN,0,0,-2,2,0.0,581,216,NaN,NaN,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,593,216,False,0.0,591,216,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,89,100.0,100.0,1,41270,NaN,NaN,0,0,-1,1,27.5,303,95,4.0,NaN,0,2,1.7,1.7,5,David,1.2,9,0,18.1,301,99,False,111.4,255,98,569,NaN,2020-12-26T18:00:15.638627Z,54,0,0,NaN,0,NaN,41270.jpg,1.6,0,0,Luiz Moreira Marinho,0.7,False,...,0.9,0.0,NaN,NaN,1.000000,0.0,0.0,1.1,0.0,0.0,0.0,3.397316,0.1,0.0,0.8,0.0,0.0,0.0,NaN,NaN,NaN,2.2,0.0,NaN,NaN,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,0.4,0.0,NaN,NaN,2.934417,1.717208,3.153215,2.513646,2.442761,0.460144,0.543411,0.318002,1.069278,0.2,0.022857
4,1,4,165,100.0,100.0,6,54694,NaN,NaN,0,0,-7,7,222.6,62,43,NaN,NaN,1,3,2.3,2.3,3,Pierre-Emerick,1.8,16,3,89.6,36,23,False,211.0,145,56,1436,NaN,2020-12-19T20:30:20.434896Z,113,1,0,1.0,0,NaN,54694.jpg,3.5,0,0,Aubameyang,7.0,False,...,0.9,0.0,0.1,0.0,7.132871,0.3,1.0,1.1,0.0,0.1,0.0,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,2.2,0.0,NaN,NaN,5.449329,0.6,0.0,0.8,1.0,0.0,0.0,6.67032,0.8,0.0,0.4,1.0,0.0,0.0,4.995843,4.370273,4.405674,3.934110,3.956641,0.358260,0.442110,0.386750,1.204309,6.1,0.381250
5,0,0,3,100.0,100.0,0,58822,NaN,NaN,0,0,-4,4,0.0,566,213,NaN,NaN,0,2,0.5,0.5,0,Cédric,0.0,0,0,0.0,581,213,False,0.0,578,212,5,NaN,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,NaN,58822.jpg,1.0,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,1.000000,0.0,0.0,1.1,0.0,0.0,0.0,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,2.2,0.0,NaN,NaN,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,0.4,0.0,NaN,NaN,NaN,1.000000,2.857447,2.243713,2.169492,0.479299,NaN,0.217391,1.027983,0.0,0.000000


In [10]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Manchester Utd,1,17,11,3,3,34,24,10,36,28.4,22.1,6.3,0.37,W D W W W,NaN,Bruno Fernandes - 11,David de Gea,NaN,1.670588,1.300000,0.272532
Liverpool,2,17,9,6,2,37,21,16,33,33.1,20.0,13.1,0.77,W W D D L,750.0,Mohamed Salah - 13,Alisson,NaN,1.947059,1.176471,0.308365
Manchester City,3,16,9,5,2,25,13,12,32,28.1,13.5,14.6,0.91,D W W W W,NaN,Riyad Mahrez Raheem Sterling... - 4,Ederson,NaN,1.756250,0.843750,0.430095
Leicester City,4,17,10,2,5,31,21,10,32,25.2,20.0,5.2,0.30,L W D D W,NaN,Jamie Vardy - 11,Kasper Schmeichel,NaN,1.482353,1.176471,0.308365
Everton,5,17,10,2,5,28,21,7,32,21.6,21.2,0.4,0.02,W W W L W,250.0,Dominic Calvert-Lewin - 11,Jordan Pickford,NaN,1.270588,1.247059,0.287349


In [11]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [12]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [13]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [14]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [15]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [16]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [17]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','elliott','mattylongstaff',"n'lundulu",'carlosvinicius']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'mattylongstaff':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.contains('matthew longstaff'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == "n'lundulu":
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('lundulu')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'carlosvinicius':
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('carlos')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [18]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,assists_week11,xPoints week 12,xG_week12,goals_week12,xGA_week12,cleansheet_week12,xA_week12,assists_week12,xPoints week 13,xG_week13,goals_week13,xGA_week13,cleansheet_week13,xA_week13,assists_week13,xPoints week 14,xG_week14,goals_week14,xGA_week14,cleansheet_week14,xA_week14,assists_week14,xPoints week 15,xG_week15,goals_week15,xGA_week15,cleansheet_week15,xA_week15,assists_week15,xPoints week 16,xG_week16,goals_week16,xGA_week16,cleansheet_week16,xA_week16,assists_week16,xPoints week 17,xG_week17,goals_week17,xGA_week17,cleansheet_week17,xA_week17,assists_week17,xPoints week 18,xG_week18,goals_week18,xGA_week18,cleansheet_week18,xA_week18,assists_week18
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-3,3,0.0,641,257,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,641,257,False,0.0,641,257,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,67,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-2,2,0.0,589,219,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,602,219,False,0.0,599,219,0,Left the club by mutual consent on 20/1,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,108,100.0,100.0,2,41270,NaN,,0,0,-1,1,28.8,312,100,4.0,,0,2,2.7,2.7,6,David,2.2,9,0,20.0,298,98,False,115.4,259,99,659,,2020-12-26T18:00:15.638627Z,54,0,0,NaN,0,,41270.jpg,2.0,0,0,Luiz Moreira Marinho,0.8,False,...,NaN,NaN,NaN,NaN,0.9,0.0,NaN,NaN,1.000000,0.0,0.0,1.1,0.0,0.0,0.0,3.397316,0.1,0.0,0.8,0.0,0.0,0.0,NaN,NaN,NaN,2.2,0.0,NaN,NaN,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,0.4,0.0,NaN,NaN,2.471518,0.0,0.0,1.0,1.0,0.0,0.0
4,1,6,202,75.0,100.0,7,54694,NaN,,0,0,-7,7,226.3,70,49,NaN,,1,3,4.0,5.3,15,Pierre-Emerick,4.8,16,5,101.8,32,22,False,272.2,114,39,1514,Personal Reasons - 75% chance of playing,2021-01-23T16:00:22.555453Z,113,1,0,1.0,0,,54694.jpg,4.2,0,0,Aubameyang,7.2,False,...,0.0,1.70657,0.2,0.0,0.9,0.0,0.1,0.0,7.132871,0.3,1.0,1.1,0.0,0.1,0.0,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,2.2,0.0,NaN,NaN,5.449329,0.6,0.0,0.8,1.0,0.0,0.0,6.67032,0.8,0.0,0.4,1.0,0.0,0.0,2.867879,0.1,0.0,1.0,1.0,0.0,0.0
5,1,3,42,100.0,100.0,1,58822,NaN,,0,0,-4,4,32.3,302,93,NaN,,0,2,2.9,2.9,12,Cédric,2.4,0,0,6.4,387,145,False,29.2,381,151,95,,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,,58822.jpg,6.5,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.0,0.0,1.1,0.0,0.0,0.0,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,2.2,0.0,NaN,NaN,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,0.4,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN


In [19]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
272,De Bruyne,16.964286,95,5.6,117.378135,6.919132
388,Kane,17.974684,142,7.9,123.108438,6.848991
302,Fernandes,18.918919,140,7.4,126.533819,6.688216
254,Salah,17.916667,129,7.2,111.721542,6.235621
390,Son,18.051948,139,7.7,109.350102,6.057524
224,Vardy,18.125000,116,6.4,108.902500,6.008414
5,Cédric,2.000000,13,6.5,11.221749,5.610875
202,Bamford,18.070175,103,5.7,99.996563,5.533790
37,Grealish,16.935484,105,6.2,92.714601,5.474576


In [20]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [21]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total goals: ' + str(total_goals))
print('Total xG: ' + str(total_xG))
print('goals per xG: ' + str(total_goals/total_xG))

Total goals: 483
Total xG: 485.0
goals per xG: 0.9958762886597938
